In [2]:
# %pip install Levenshtein
import numpy as np
import pandas as pd
import Levenshtein as lev
from itertools import product

Loading Data

In [43]:
sevMil = pd.read_csv("companies_sorted-1.csv")
ft = pd.read_csv("Forbes_top 2000_2022.csv")
db = pd.read_csv("sparql_2022-10-02_13-24-07Z.csv")

Creating ID

In [44]:
sevMil = sevMil[['name']]
sevMil['id'] =  range(len(sevMil))
sevMil['id'] = "7.1M_" + sevMil['id'].astype(str)

In [45]:
ft = ft[['organizationName']]
ft['id'] =  range(len(ft))
ft['id'] = "ft_" + ft['id'].astype(str)

In [46]:
dbpedia = db[['name']]
dbpedia['id'] =  range(len(db))
dbpedia['id'] = "db_" + dbpedia['id'].astype(str)

<ipython-input-46-49af2a1eed77>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dbpedia['id'] =  range(len(db))
<ipython-input-46-49af2a1eed77>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dbpedia['id'] = "db_" + dbpedia['id'].astype(str)


Calculating Levenshtein Distance

For FT and DBPedia Datasets

In [49]:
# Calculate the distance (ratio)
ft_db_lev = pd.DataFrame(product(ft['organizationName'], dbpedia['name']), columns=["ft","dbpedia"])
ft_db_lev["LevScore"] = ft_db_lev.apply(lambda x: lev.ratio(x[0],x[1]), axis=1)

# Filter similiar pairs 
ft_db_lev_filtered = ft_db_lev[ft_db_lev['LevScore'] > 0.8]

# Identify matching and similiar-but-unmatching pairs 
confirmed_true_ftdb = ft_db_lev_filtered[ft_db_lev_filtered['LevScore'] == 1]
edge_cases_ftdb = ft_db_lev_filtered[ft_db_lev_filtered['LevScore'] != 1]

# Sample 200 for output
true_sample = confirmed_true_ftdb.sample(200)
false_sample = edge_cases_ftdb.sample(200)

# Goldstandard output
gold_standard_ft_db = pd.concat([true_sample,false_sample])
gold_standard_ft_db = gold_standard_ft_db.merge(ft, how='left', left_on='ft', right_on='organizationName')
gold_standard_ft_db.rename(columns={'id':'ft_id'}, inplace = True)
gold_standard_ft_db = gold_standard_ft_db.merge(dbpedia, how = 'left', left_on = 'dbpedia', right_on = 'name')
gold_standard_ft_db.rename(columns={'id':'db_id'}, inplace = True)
gold_standard_ft_db = gold_standard_ft_db[['ft_id', 'db_id', 'LevScore', 'ft', 'dbpedia']]
gold_standard_ft_db['label'] = gold_standard_ft_db.apply(lambda x: True if x['LevScore'] == 1 else False, axis = 1)
gold_standard_ft_db

,ft_id,db_id,LevScore,ft,dbpedia,label
0,ft_1295,db_8255,1.000000,Ametek,Ametek,True
1,ft_1203,db_8265,1.000000,Beiersdorf,Beiersdorf,True
2,ft_40,db_4396,1.000000,Mercedes-Benz Group,Mercedes-Benz Group,True
3,ft_1470,db_9761,1.000000,Charoen Pokphand Foods,Charoen Pokphand Foods,True
4,ft_1153,db_8370,1.000000,Bank Negara Indonesia,Bank Negara Indonesia,True
...,...,...,...,...,...,...
406,ft_806,db_7757,0.833333,T&D Holdings,JAT Holdings,False
407,ft_1962,db_2551,0.846154,Mashreq Bank,Mashreq (bank),False
408,ft_801,db_3325,0.888889,US Steel,U.S. Steel,False
409,ft_256,db_8605,0.820513,KB Financial Group,South Financial Group,False


For FT and 7Mil datasets

In [54]:
# Calculate the distance (ratio)
ft_sevM_lev = pd.DataFrame(product(ft['organizationName'], sevMil['name']), columns=["ft","sevMil"])
ft_sevM_lev["LevScore"] = ft_sevM_lev.apply(lambda x: lev.ratio(x[0],x[1]), axis=1)

# Filter similiar pairs 
ft_sevM_lev_filtered = ft_sevM_lev[ft_sevM_lev['LevScore'] > 0.8]

# Identify matching and similiar-but-unmatching pairs
confirmed_true_ftsevM = ft_sevM_lev_filtered[ft_sevM_lev_filtered['LevScore'] == 1]
edge_cases_ftsevM = ft_sevM_lev_filtered[ft_sevM_lev_filtered['LevScore'] != 1]

# Sample 200 for output
# true_sample = confirmed_true_ftsevM.sample(200)
true_sample = confirmed_true_ftsevM
false_sample = edge_cases_ftsevM.sample(200)

# Goldstandard output
gold_standard_ftsevM = pd.concat([true_sample,false_sample])
gold_standard_ftsevM = gold_standard_ftsevM.merge(ft, how='left', left_on='ft', right_on='organizationName')
gold_standard_ftsevM.rename(columns={'id':'ft_id'}, inplace = True)
gold_standard_ftsevM = gold_standard_ftsevM.merge(sevMil, how = 'left', left_on = 'sevMil', right_on = 'name')
gold_standard_ftsevM.rename(columns={'id':'sevM_id'}, inplace = True)
gold_standard_ftsevM = gold_standard_ftsevM[['ft_id', 'sevM_id', 'LevScore', 'ft', 'sevMil']]
gold_standard_ftsevM['label'] = gold_standard_ftsevM.apply(lambda x: True if x['LevScore'] == 1 else False, axis = 1)
gold_standard_ftsevM

,ft_id,sevM_id,LevScore,ft,sevMil,label
0,ft_866,7.1M_1362,0.866667,Bank of Ireland,bank of ireland,False
1,ft_779,7.1M_1731,0.866667,Astellas Pharma,astellas pharma,False
2,ft_310,7.1M_693,0.866667,Macquarie Group,macquarie group,False
3,ft_951,7.1M_803,0.888889,Motorola Solutions,motorola solutions,False
4,ft_1097,7.1M_1046,0.823529,Dassault Systemes,dassault systèmes,False
...,...,...,...,...,...,...
198,ft_274,7.1M_1616,0.818182,Volvo Group,volvo group,False
199,ft_583,7.1M_458,0.823529,Colgate-Palmolive,colgate palmolive,False
200,ft_484,7.1M_1525,0.857143,Maybank,maybank,False
201,ft_96,7.1M_195,0.850000,Lloyds Banking Group,lloyds banking group,False
